In [ ]:
!pip install timm foolbox

     |████████████████████████████████| 296kB 5.8MB/s 
     |████████████████████████████████| 1.7MB 12.6MB/s 
     |████████████████████████████████| 61kB 10.6MB/s 
     |████████████████████████████████| 163kB 47.0MB/s 
     |████████████████████████████████| 71kB 11.4MB/s 
ERROR: google-colab 1.0.0 has requirement requests~=2.23.0, but you'll have requests 2.25.1 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
  Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0


In [ ]:
#@title Mount Drive
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
#@title Imports

import eagerpy as ep
from foolbox import PyTorchModel, accuracy, samples
from foolbox.attacks import L2PGD

import timm

from __future__ import print_function
from __future__ import division
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
print("PyTorch Version: ",torch.__version__)
print("Torchvision Version: ",torchvision.__version__)

from tqdm import tqdm

import cv2

import matplotlib.pyplot as plt

PyTorch Version:  1.8.1+cu101
Torchvision Version:  0.9.1+cu101


In [ ]:
#@title Global Variables

# Number of classes in the dataset
num_classes = 2
batch_size = 32
input_size = 224
feature_extract = False
num_epochs = 5
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
#@title Model
model = models.resnet18(pretrained=False)
num_ftr = model.fc.in_features
model.fc = nn.Linear(num_ftr, num_classes)

# load weights
model.load_state_dict(torch.load('/content/drive/MyDrive/598dataset/Detector_ResNet50.pth'))

model.to(device)
model.eval()

In [ ]:
class fft(object):
  def __call__(self, sample):
    sample = torch.fft.fft2(sample)
    sample = torch.fft.fftshift(sample,dim=(-2,-1))
    return sample.float()

#@title Dataloader Init
transform = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(input_size),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
        fft()
    ]),
    'val': transforms.Compose([
        transforms.Resize(input_size),
        transforms.CenterCrop(input_size),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
        fft()
    ]),
}


data_dir = '/content/drive/MyDrive/598dataset/tinyImageNet/TinyImageNet'
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, "tiny_"+x),
                                          transform[x])
                  for x in ['train', 'val']}
                  
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=32,
                                             shuffle=True, num_workers=4, pin_memory=True)
              for x in ['train', 'val']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
class_names = image_datasets['train'].classes
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# get some constants
NUM_CLASSES = len(class_names)

counter = 0
correct = 0
with tqdm(dataloaders['val'], unit='batch') as tepoch:
  for inputs, labels in tepoch:
    inputs = inputs.to(device)
    labels = labels.to(device)
    pred = model(inputs).argmax(axis=1)
    # print(pred.shape, inputs.shape)
    # print(pred)
    correct +=  pred.sum()
    counter += pred.shape[0]

print(f"clean accuracy:  {correct/counter * 100:.1f} %")

In [ ]:
# counter = 0
# correct = 0
# with tqdm(dataloaders['val'], unit='batch') as tepoch:
#   for inputs, labels in tepoch:
#     inputs = inputs.to(device)
#     labels = labels.to(device)
#     pred = model(inputs).argmax(axis=1)
#     # print(pred.shape, inputs.shape)
#     # print(pred)
#     correct +=  pred.sum()
#     counter += pred.shape[0]

# print(f"clean accuracy:  {correct/counter * 100:.1f} %")

  0%|          | 0/50 [00:00<?, ?batch/s]/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: UserWarning: Casting complex values to real discards the imaginary part (Triggered internally at  /pytorch/aten/src/ATen/native/Copy.cpp:219.)
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: UserWarning: Casting complex values to real discards the imaginary part (Triggered internally at  /pytorch/aten/src/ATen/native/Copy.cpp:219.)
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: UserWarning: Casting complex values to real discards the imaginary part (Triggered internally at  /pytorch/aten/src/ATen/native/Copy.cpp:219.)
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: UserWarning: Casting complex values to real discards the imaginary part (Triggered internally at  /pytorch/aten/src/ATen/native/Copy.cpp:219.)
  """
100%|██████████| 50/50 [01:22<00:00,  1.65s/batch]

clean accuracy:  100.0 %


In [ ]:
class fft(object):
  def __call__(self, sample):
    sample = torch.fft.fft2(sample)
    sample = torch.fft.fftshift(sample,dim=(-2,-1))
    return sample.float()

#@title Dataloader Init
transform = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(input_size),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
        fft()
    ]),
    'val': transforms.Compose([
        transforms.Resize(input_size),
        transforms.CenterCrop(input_size),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
        fft()
    ]),
}


data_dir = '/content/drive/MyDrive/598dataset/TinyImgNet_AdvSamples/VIT/PGD_L2/Epsilon0.5'
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          transform[x])
                  for x in ['train', 'val']}
                  
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=32,
                                             shuffle=True, num_workers=4, pin_memory=True)
              for x in ['train', 'val']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
class_names = image_datasets['train'].classes
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# get some constants
NUM_CLASSES = len(class_names)

counter = 0
correct = 0
with tqdm(dataloaders['val'], unit='batch') as tepoch:
  for inputs, labels in tepoch:
    inputs = inputs.to(device)
    labels = labels.to(device)
    pred = model(inputs).argmax(axis=1)
    # print(pred.shape, inputs.shape)
    # print(pred)
    correct +=  pred.shape[0] - pred.sum()
    counter += pred.shape[0]

print(f"clean accuracy:  {correct/counter * 100:.1f} %")

  0%|          | 0/50 [00:00<?, ?batch/s]/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: UserWarning: Casting complex values to real discards the imaginary part (Triggered internally at  /pytorch/aten/src/ATen/native/Copy.cpp:219.)
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: UserWarning: Casting complex values to real discards the imaginary part (Triggered internally at  /pytorch/aten/src/ATen/native/Copy.cpp:219.)
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: UserWarning: Casting complex values to real discards the imaginary part (Triggered internally at  /pytorch/aten/src/ATen/native/Copy.cpp:219.)
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: UserWarning: Casting complex values to real discards the imaginary part (Triggered internally at  /pytorch/aten/src/ATen/native/Copy.cpp:219.)
  """
100%|██████████| 50/50 [00:04<00:00, 10.34batch/s]

clean accuracy:  100.0 %


In [ ]:
#@title test C&W 0.5

class fft(object):
  def __call__(self, sample):
    sample = torch.fft.fft2(sample)
    sample = torch.fft.fftshift(sample,dim=(-2,-1))
    return sample.float()

#@title Dataloader Init
transform = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(input_size),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
        fft()
    ]),
    'val': transforms.Compose([
        transforms.Resize(input_size),
        transforms.CenterCrop(input_size),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
        fft()
    ]),
}


data_dir = '/content/drive/MyDrive/598dataset/TinyImgNet_AdvSamples/VIT/C&W_L2/Epsilon0.5'
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          transform[x])
                  for x in ['train', 'val']}
                  
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=32,
                                             shuffle=True, num_workers=4, pin_memory=True)
              for x in ['train', 'val']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
class_names = image_datasets['train'].classes
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# get some constants
NUM_CLASSES = len(class_names)

counter = 0
correct = 0
with tqdm(dataloaders['val'], unit='batch') as tepoch:
  for inputs, labels in tepoch:
    inputs = inputs.to(device)
    labels = labels.to(device)
    pred = model(inputs).argmax(axis=1)
    # print(pred.shape, inputs.shape)
    # print(pred)
    correct +=  pred.shape[0] - pred.sum()
    counter += pred.shape[0]

print(f"clean accuracy:  {correct/counter * 100:.1f} %")

  0%|          | 0/50 [00:00<?, ?batch/s]/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: UserWarning: Casting complex values to real discards the imaginary part (Triggered internally at  /pytorch/aten/src/ATen/native/Copy.cpp:219.)
  import sys
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: UserWarning: Casting complex values to real discards the imaginary part (Triggered internally at  /pytorch/aten/src/ATen/native/Copy.cpp:219.)
  import sys
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: UserWarning: Casting complex values to real discards the imaginary part (Triggered internally at  /pytorch/aten/src/ATen/native/Copy.cpp:219.)
  import sys
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: UserWarning: Casting complex values to real discards the imaginary part (Triggered internally at  /pytorch/aten/src/ATen/native/Copy.cpp:219.)
  import sys
100%|██████████| 50/50 [04:42<00:00,  5.65s/batch]

clean accuracy:  86.5 %


In [ ]:
#@title test PGD-L2 Epsilon=2

class fft(object):
  def __call__(self, sample):
    sample = torch.fft.fft2(sample)
    sample = torch.fft.fftshift(sample,dim=(-2,-1))
    return sample.float()

#@title Dataloader Init
transform = {
    'val': transforms.Compose([
        transforms.Resize(input_size),
        transforms.CenterCrop(input_size),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
        fft()
    ]),
}


data_dir = '/content/drive/MyDrive/598dataset/TinyImgNet_AdvSamples/VIT/PGD_L2/Epsilon2'
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          transform[x])
                  for x in ['val']}
                  
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=32,
                                             shuffle=True, num_workers=4, pin_memory=True)
              for x in ['val']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['val']}
class_names = image_datasets['val'].classes
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# get some constants
NUM_CLASSES = len(class_names)

counter = 0
correct = 0
with tqdm(dataloaders['val'], unit='batch') as tepoch:
  for inputs, labels in tepoch:
    inputs = inputs.to(device)
    labels = labels.to(device)
    pred = model(inputs).argmax(axis=1)
    # print(pred.shape, inputs.shape)
    # print(pred)
    correct +=  pred.shape[0] - pred.sum()
    counter += pred.shape[0]

print(f"clean accuracy:  {correct/counter * 100:.1f} %")

  0%|          | 0/50 [00:00<?, ?batch/s]/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: UserWarning: Casting complex values to real discards the imaginary part (Triggered internally at  /pytorch/aten/src/ATen/native/Copy.cpp:219.)
  import sys
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: UserWarning: Casting complex values to real discards the imaginary part (Triggered internally at  /pytorch/aten/src/ATen/native/Copy.cpp:219.)
  import sys
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: UserWarning: Casting complex values to real discards the imaginary part (Triggered internally at  /pytorch/aten/src/ATen/native/Copy.cpp:219.)
  import sys
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: UserWarning: Casting complex values to real discards the imaginary part (Triggered internally at  /pytorch/aten/src/ATen/native/Copy.cpp:219.)
  import sys
100%|██████████| 50/50 [04:03<00:00,  4.86s/batch]

clean accuracy:  82.6 %


In [ ]:
#@title test PGD-L2 Epsilon=0.3

class fft(object):
  def __call__(self, sample):
    sample = torch.fft.fft2(sample)
    sample = torch.fft.fftshift(sample,dim=(-2,-1))
    return sample.float()

#@title Dataloader Init
transform = {
    'val': transforms.Compose([
        transforms.Resize(input_size),
        transforms.CenterCrop(input_size),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
        fft()
    ]),
}


data_dir = '/content/drive/MyDrive/598dataset/TinyImgNet_AdvSamples/VIT/PGD_L2/Epsilon0.3'
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          transform[x])
                  for x in ['val']}
                  
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=32,
                                             shuffle=True, num_workers=4, pin_memory=True)
              for x in ['val']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['val']}
class_names = image_datasets['val'].classes
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# get some constants
NUM_CLASSES = len(class_names)

counter = 0
correct = 0
with tqdm(dataloaders['val'], unit='batch') as tepoch:
  for inputs, labels in tepoch:
    inputs = inputs.to(device)
    labels = labels.to(device)
    pred = model(inputs).argmax(axis=1)
    # print(pred.shape, inputs.shape)
    # print(pred)
    correct +=  pred.shape[0] - pred.sum()
    counter += pred.shape[0]

print(f"clean accuracy:  {correct/counter * 100:.1f} %")

  0%|          | 0/50 [00:00<?, ?batch/s]/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: UserWarning: Casting complex values to real discards the imaginary part (Triggered internally at  /pytorch/aten/src/ATen/native/Copy.cpp:219.)
  import sys
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: UserWarning: Casting complex values to real discards the imaginary part (Triggered internally at  /pytorch/aten/src/ATen/native/Copy.cpp:219.)
  import sys
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: UserWarning: Casting complex values to real discards the imaginary part (Triggered internally at  /pytorch/aten/src/ATen/native/Copy.cpp:219.)
  import sys
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: UserWarning: Casting complex values to real discards the imaginary part (Triggered internally at  /pytorch/aten/src/ATen/native/Copy.cpp:219.)
  import sys
100%|██████████| 50/50 [05:41<00:00,  6.82s/batch]

clean accuracy:  100.0 %


In [ ]:
#@title test PGD-L2 Epsilon=0.1

class fft(object):
  def __call__(self, sample):
    sample = torch.fft.fft2(sample)
    sample = torch.fft.fftshift(sample,dim=(-2,-1))
    return sample.float()

#@title Dataloader Init
transform = {
    'val': transforms.Compose([
        transforms.Resize(input_size),
        transforms.CenterCrop(input_size),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
        fft()
    ]),
}


data_dir = '/content/drive/MyDrive/598dataset/TinyImgNet_AdvSamples/VIT/PGD_L2/Epsilon0.1'
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          transform[x])
                  for x in ['val']}
                  
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=32,
                                             shuffle=True, num_workers=4, pin_memory=True)
              for x in ['val']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['val']}
class_names = image_datasets['val'].classes
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# get some constants
NUM_CLASSES = len(class_names)

counter = 0
correct = 0
with tqdm(dataloaders['val'], unit='batch') as tepoch:
  for inputs, labels in tepoch:
    inputs = inputs.to(device)
    labels = labels.to(device)
    pred = model(inputs).argmax(axis=1)
    # print(pred.shape, inputs.shape)
    # print(pred)
    correct +=  pred.shape[0] - pred.sum()
    counter += pred.shape[0]

print(f"clean accuracy:  {correct/counter * 100:.1f} %")

  0%|          | 0/50 [00:00<?, ?batch/s]/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: UserWarning: Casting complex values to real discards the imaginary part (Triggered internally at  /pytorch/aten/src/ATen/native/Copy.cpp:219.)
  import sys
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: UserWarning: Casting complex values to real discards the imaginary part (Triggered internally at  /pytorch/aten/src/ATen/native/Copy.cpp:219.)
  import sys
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: UserWarning: Casting complex values to real discards the imaginary part (Triggered internally at  /pytorch/aten/src/ATen/native/Copy.cpp:219.)
  import sys
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: UserWarning: Casting complex values to real discards the imaginary part (Triggered internally at  /pytorch/aten/src/ATen/native/Copy.cpp:219.)
  import sys
100%|██████████| 50/50 [05:23<00:00,  6.48s/batch]

clean accuracy:  100.0 %


In [ ]:
#@title test FGSM-L2 Epsilon=0.1

class fft(object):
  def __call__(self, sample):
    sample = torch.fft.fft2(sample)
    sample = torch.fft.fftshift(sample,dim=(-2,-1))
    return sample.float()

#@title Dataloader Init
transform = {
    'val': transforms.Compose([
        transforms.Resize(input_size),
        transforms.CenterCrop(input_size),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
        fft()
    ]),
}


data_dir = '/content/drive/MyDrive/598dataset/TinyImgNet_AdvSamples/VIT/FGSM/Epsilon0.1'
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          transform[x])
                  for x in ['val']}
                  
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=32,
                                             shuffle=True, num_workers=4, pin_memory=True)
              for x in ['val']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['val']}
class_names = image_datasets['val'].classes
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# get some constants
NUM_CLASSES = len(class_names)

counter = 0
correct = 0
with tqdm(dataloaders['val'], unit='batch') as tepoch:
  for inputs, labels in tepoch:
    inputs = inputs.to(device)
    labels = labels.to(device)
    pred = model(inputs).argmax(axis=1)
    # print(pred.shape, inputs.shape)
    # print(pred)
    correct +=  pred.shape[0] - pred.sum()
    counter += pred.shape[0]

print(f"clean accuracy:  {correct/counter * 100:.1f} %")

  0%|          | 0/50 [00:00<?, ?batch/s]/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: UserWarning: Casting complex values to real discards the imaginary part (Triggered internally at  /pytorch/aten/src/ATen/native/Copy.cpp:219.)
  import sys
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: UserWarning: Casting complex values to real discards the imaginary part (Triggered internally at  /pytorch/aten/src/ATen/native/Copy.cpp:219.)
  import sys
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: UserWarning: Casting complex values to real discards the imaginary part (Triggered internally at  /pytorch/aten/src/ATen/native/Copy.cpp:219.)
  import sys
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: UserWarning: Casting complex values to real discards the imaginary part (Triggered internally at  /pytorch/aten/src/ATen/native/Copy.cpp:219.)
  import sys
100%|██████████| 50/50 [05:33<00:00,  6.67s/batch]

clean accuracy:  0.0 %


In [ ]:
#@title test FGSM-L2 Epsilon=0.01

class fft(object):
  def __call__(self, sample):
    sample = torch.fft.fft2(sample)
    sample = torch.fft.fftshift(sample,dim=(-2,-1))
    return sample.float()

#@title Dataloader Init
transform = {
    'val': transforms.Compose([
        transforms.Resize(input_size),
        transforms.CenterCrop(input_size),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
        fft()
    ]),
}


data_dir = '/content/drive/MyDrive/598dataset/TinyImgNet_AdvSamples/VIT/FGSM/Epsilon0.01'
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          transform[x])
                  for x in ['val']}
                  
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=32,
                                             shuffle=True, num_workers=4, pin_memory=True)
              for x in ['val']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['val']}
class_names = image_datasets['val'].classes
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# get some constants
NUM_CLASSES = len(class_names)

counter = 0
correct = 0
with tqdm(dataloaders['val'], unit='batch') as tepoch:
  for inputs, labels in tepoch:
    inputs = inputs.to(device)
    labels = labels.to(device)
    pred = model(inputs).argmax(axis=1)
    # print(pred.shape, inputs.shape)
    # print(pred)
    correct +=  pred.shape[0] - pred.sum()
    counter += pred.shape[0]

print(f"clean accuracy:  {correct/counter * 100:.1f} %")

  0%|          | 0/50 [00:00<?, ?batch/s]/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: UserWarning: Casting complex values to real discards the imaginary part (Triggered internally at  /pytorch/aten/src/ATen/native/Copy.cpp:219.)
  import sys
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: UserWarning: Casting complex values to real discards the imaginary part (Triggered internally at  /pytorch/aten/src/ATen/native/Copy.cpp:219.)
  import sys
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: UserWarning: Casting complex values to real discards the imaginary part (Triggered internally at  /pytorch/aten/src/ATen/native/Copy.cpp:219.)
  import sys
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: UserWarning: Casting complex values to real discards the imaginary part (Triggered internally at  /pytorch/aten/src/ATen/native/Copy.cpp:219.)
  import sys
100%|██████████| 50/50 [06:01<00:00,  7.23s/batch]

clean accuracy:  0.0 %


In [ ]:
#@title test pgd-Linf Epsilon=0.01

class fft(object):
  def __call__(self, sample):
    sample = torch.fft.fft2(sample)
    sample = torch.fft.fftshift(sample,dim=(-2,-1))
    return sample.float()

#@title Dataloader Init
transform = {
    'val': transforms.Compose([
        transforms.Resize(input_size),
        transforms.CenterCrop(input_size),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
        fft()
    ]),
}


data_dir = '/content/drive/MyDrive/598dataset/TinyImgNet_AdvSamples/VIT/PGD_Linf/Epsilon0.01'
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          transform[x])
                  for x in ['val']}
                  
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=32,
                                             shuffle=True, num_workers=4, pin_memory=True)
              for x in ['val']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['val']}
class_names = image_datasets['val'].classes
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# get some constants
NUM_CLASSES = len(class_names)

counter = 0
correct = 0
with tqdm(dataloaders['val'], unit='batch') as tepoch:
  for inputs, labels in tepoch:
    inputs = inputs.to(device)
    labels = labels.to(device)
    pred = model(inputs).argmax(axis=1)
    # print(pred.shape, inputs.shape)
    # print(pred)
    correct +=  pred.shape[0] - pred.sum()
    counter += pred.shape[0]

print(f"clean accuracy:  {correct/counter * 100:.1f} %")

  0%|          | 0/50 [00:00<?, ?batch/s]/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: UserWarning: Casting complex values to real discards the imaginary part (Triggered internally at  /pytorch/aten/src/ATen/native/Copy.cpp:219.)
  import sys
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: UserWarning: Casting complex values to real discards the imaginary part (Triggered internally at  /pytorch/aten/src/ATen/native/Copy.cpp:219.)
  import sys
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: UserWarning: Casting complex values to real discards the imaginary part (Triggered internally at  /pytorch/aten/src/ATen/native/Copy.cpp:219.)
  import sys
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: UserWarning: Casting complex values to real discards the imaginary part (Triggered internally at  /pytorch/aten/src/ATen/native/Copy.cpp:219.)
  import sys
100%|██████████| 50/50 [04:43<00:00,  5.68s/batch]

clean accuracy:  1.8 %


In [ ]:
#@title test pgd-Linf Epsilon=0.005

class fft(object):
  def __call__(self, sample):
    sample = torch.fft.fft2(sample)
    sample = torch.fft.fftshift(sample,dim=(-2,-1))
    return sample.float()

#@title Dataloader Init
transform = {
    'val': transforms.Compose([
        transforms.Resize(input_size),
        transforms.CenterCrop(input_size),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
        fft()
    ]),
}


data_dir = '/content/drive/MyDrive/598dataset/TinyImgNet_AdvSamples/VIT/PGD_Linf/Epsilon0.005'
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          transform[x])
                  for x in ['val']}
                  
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=32,
                                             shuffle=True, num_workers=4, pin_memory=True)
              for x in ['val']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['val']}
class_names = image_datasets['val'].classes
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# get some constants
NUM_CLASSES = len(class_names)

counter = 0
correct = 0
with tqdm(dataloaders['val'], unit='batch') as tepoch:
  for inputs, labels in tepoch:
    inputs = inputs.to(device)
    labels = labels.to(device)
    pred = model(inputs).argmax(axis=1)
    # print(pred.shape, inputs.shape)
    # print(pred)
    correct +=  pred.shape[0] - pred.sum()
    counter += pred.shape[0]

print(f"clean accuracy:  {correct/counter * 100:.1f} %")

  0%|          | 0/50 [00:00<?, ?batch/s]/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: UserWarning: Casting complex values to real discards the imaginary part (Triggered internally at  /pytorch/aten/src/ATen/native/Copy.cpp:219.)
  import sys
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: UserWarning: Casting complex values to real discards the imaginary part (Triggered internally at  /pytorch/aten/src/ATen/native/Copy.cpp:219.)
  import sys
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: UserWarning: Casting complex values to real discards the imaginary part (Triggered internally at  /pytorch/aten/src/ATen/native/Copy.cpp:219.)
  import sys
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: UserWarning: Casting complex values to real discards the imaginary part (Triggered internally at  /pytorch/aten/src/ATen/native/Copy.cpp:219.)
  import sys
100%|██████████| 50/50 [04:30<00:00,  5.41s/batch]

clean accuracy:  100.0 %
